In [ ]:
# FRIENDLY REMINDER: RUNTIME > 1 HOUR FOR THE ENTIRE PROCESS

In [ ]:
import pymysql
import pandas as pd
import datetime
import json
import time as timer
from tqdm import tqdm

In [ ]:
#Get data from SQL and return to dataframe
def data_out_sql(sql):
    conn = pymysql.connect(host='localhost', user='root', password='thisisnotmypw', database="Twitter",charset='utf8')
    cursor = conn.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()
    cols = cursor.description
    conn.commit()
    conn.close()
    #transform sql to dataframe
    col = []
    for i in cols:
        col.append(i[0])
    data = list(map(list, data))
    data = pd.DataFrame(data,columns=col)
    return data


In [ ]:
# target tickers
w = open("ticker.txt",'r')
l = w.readlines()
tickers=[]
total = {}
for k in l:
    k = k.strip('\n')
    tickers.append('$'+k)

In [ ]:
# import training data from SQL
sql = "select ticker,created_time,text,retweets,replies,likes,is_retweet from clean_train"  
twitter_data_train = data_out_sql(sql)

In [ ]:
# datetime transform
twitter_data_train['created_time'] = twitter_data_train['created_time'].apply(lambda x: x[:10])
twitter_data_train['created_time'] = pd.to_datetime(twitter_data_train['created_time'])

In [ ]:
# read price data
price = pd.read_csv("close&eps_171819.csv")

In [ ]:
# mutate string to datetime type
start_time = timer.time()
price['datadate']= price.apply(lambda x: pd.to_datetime(x[2], format='%Y%m%d'),axis=1)
# time of running
end_time = timer.time()
print(end_time-start_time,'seconds')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/Social Media Group /Data/clean4_test 2.csv')

In [ ]:
test

,Unnamed: 0,level_0,index,ticker,created_time,text,retweets,replies,likes,is_retweet
0,0,0,0,$PLUG,2019-06-06,big question collaboration team apart project,0,5,5,0
1,1,1,1,$PLUG,2019-06-06,plug power stock could bargain,0,0,0,0
2,2,2,2,$PLUG,2019-06-06,glad stay away one well company,0,0,0,0
3,3,3,3,$PLUG,2019-06-06,tech stock far year via go team,0,0,2,0
4,4,4,4,$PLUG,2019-06-06,go team,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...
1953273,1953273,1953273,1953273,$WLL,2019-12-25,long,0,2,0,0
1953274,1953274,1953274,1953274,$WLL,2019-12-25,short sale volume short interest gold ng,0,0,0,0
1953275,1953275,1953275,1953275,$WLL,2019-12-30,gap production forecast performance grow,1,0,1,0
1953276,1953276,1953276,1953276,$WLL,2019-12-30,gap production forecast performance grow,0,0,1,0


In [ ]:
# Consolidation
def concat_func(x):
#     return pd.Series({
#         'text':' '.join(x['text']),
    return ' '.join(x.values)
#     })


In [ ]:
test = test.groupby(['ticker','created_time'])['text'].apply(concat_func).reset_index()

TypeError: ignored

In [ ]:
# Consolidation
def concat_func(x):
#     return pd.Series({
#         'text':' '.join(x['text']),
    return ' '.join(x.values)
#     })

# consolidate tweets
twitter_data_train_consolid = twitter_data_train.copy()
twitter_data_train_consolid = twitter_data_train_consolid.groupby(['ticker','created_time'])['text'].apply(concat_func).reset_index()

# sum up number of retweets, replies, likes, is_retweet
tmp = twitter_data_train.copy()
twitter_data_train_consolid = twitter_data_train_consolid.join(tmp.groupby(['ticker','created_time'])['retweets'].apply(lambda x:sum(x)).reset_index()['retweets'])
twitter_data_train_consolid = twitter_data_train_consolid.join(tmp.groupby(['ticker','created_time'])['replies'].apply(lambda x:sum(x)).reset_index()['replies'])
twitter_data_train_consolid = twitter_data_train_consolid.join(tmp.groupby(['ticker','created_time'])['likes'].apply(lambda x:sum(x)).reset_index()['likes'])
twitter_data_train_consolid = twitter_data_train_consolid.join(tmp.groupby(['ticker','created_time'])['is_retweet'].apply(lambda x:sum(x)).reset_index()['is_retweet'])


In [ ]:
# filter out posts by ticker and store in a dict in order
def filter_ticker(twitter_data_train):
    start_time = timer.time()
    twitter_dict = dict()
    for tic in tickers:
        try:
            data = twitter_data_train.loc[twitter_data_train['ticker']==tic]
            data = data.reset_index()
            data = data.drop(columns = 'index')
            twitter_dict[tic] = data
            print(tic)
        except:
            empty_df = pd.DataFrame(
                columns = ['ticker', 'created_time', 'text', 'retweets', 'replies', 'likes', 'is_retweet'])
            twitter_dict[tic] = empty_df
            print(tic)
        
    # time of running
    end_time = timer.time()
    print(end_time-start_time,'seconds')
    
    return twitter_dict

In [ ]:
twitter_dict = filter_ticker(twitter_data_train_consolid)

In [ ]:
# return a df with price and related tweets
def get_tweets(price,twitter_dict,twitter_day_diff):
# combine stock info and tweets
# create a list as the 'tweets' column
    start_time = timer.time()
    time_time = timer.time()

    twitter = list()
    #twitter_day_diff = 3

    # set the len of stock df (price) as range
    for index in range(len(price)):
        tic = '$'+price.iloc[index]['tic']
        time = price.iloc[index]['datadate']
        if tic in twitter_dict:
            tmp = twitter_dict[tic].copy()
            ### set target time period - Use + datetime.timedelta(days=) to Add/Subtract Days
            lbound_date = time - datetime.timedelta(days=twitter_day_diff)
            ubound_date = time
            tmp = tmp.loc[(tmp['created_time']>=lbound_date) & (tmp['created_time']<ubound_date)]
            tmp = tmp.reset_index()
            ###
            twitter.append(tmp)
        else:
            empty_df = pd.DataFrame(
                columns = ['ticker', 'created_time', 'text', 'retweets', 'replies', 'likes', 'is_retweet'])
            twitter.append(empty_df)
    
        # print time
        if (index+1)%10000 == 0:
            time_diff = timer.time()-time_time
            print("Just finished: {} -- Time: {}s".format(index+1,time_diff))
            time_time = timer.time()
    
    # combine the list with the 'price' df
    price_twitter = price.copy()
    price_twitter['tweets'] = twitter
    price_twitter = price_twitter.reset_index()
    price_twitter = price_twitter.drop(columns = 'index')
    
    # time of running
    end_time = timer.time()
    print(end_time-start_time,'seconds')
    
    return price_twitter

In [ ]:
##### get 3-day tweets (to match the close price later)
price_twitter = get_tweets(price,twitter_dict,3)

In [ ]:
price_twitter[price_twitter['tic']=="VERI"]

In [ ]:
### get percent change of close price
def percent_change(price_twitter,tic,day_diff):
    #try stock: RMBS
    #tic = "RMBS"
    TIC_price = price_twitter.loc[price_twitter['tic']==tic].copy()

    #set n_day_change
    #day_diff = 3

    tmp = list()
    for index in range(len(TIC_price)):
        if index < day_diff:
            # the first few data point has no n_day_change
            tmp.append(-100000000)
        else:
            today_price = TIC_price['prccd'].iloc[index]
            previous_price = TIC_price['prccd'].iloc[index-day_diff]
            diff_percent = (today_price-previous_price)/previous_price
            tmp.append(diff_percent)
        
    #this is the column name of the new n_day_change column
    col_name = str(day_diff)+'_day_change'
    #join
    TIC_price[col_name] = tmp 
    return TIC_price

In [ ]:
# get n_day_change for all stocks, store in a dict
price_twitter_pchange = dict()
for tic in tqdm(tickers):
    startt = timer.time()
    # get rid of the $ sign
    tic = tic[1:]
    #print("Processing {}".format(tic))
    # get the n_day_change filter by tic
    TIC_price = percent_change(price_twitter,tic,3)
    # store into the dictionary
    price_twitter_pchange[tic] = TIC_price
    #print("{}s".format(timer.time()-startt))

In [ ]:
price_twitter_pchange['VERI']

In [ ]:
col_name = '3_day_change'
TIC_price = price_twitter_pchange['VERI']
TIC_price.loc[TIC_price[col_name]>=0.1]